In [2]:
import sys
sys.path.append("/home/jittat/prog/django/admapp")
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django_bootstrap import bootstrap
bootstrap()

from django.core import serializers
import json

In [3]:
from criteria.models import *

In [4]:
project_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 50]

In [14]:
criterias = {}

for cupt_code in list(MajorCuptCode.objects.all()):
    data1 = serializers.serialize('json',[cupt_code])
    data2 = json.loads(data1)[0]['fields']
    
    criterias[(cupt_code.program_code,cupt_code.major_code)] = {
        'data': {
            'major_cupt_code': data2,
            'criterias': {},
        },
        'objects': {
            'cupt_code': cupt_code,
            'criterias': {},
        }
    }
    
print(criterias)

{('10020101111802A', ''): {'data': {'major_cupt_code': {'program_code': '10020101111802A', 'program_type': 'ภาษาไทย ปกติ', 'program_type_code': 'A', 'faculty': 2, 'major_code': '', 'title': 'วท.บ. สาขาวิชาอาหาร โภชนาการ และการกำหนดอาหาร', 'major_title': '', 'component_weight_type': ''}, 'criterias': {}}, 'objects': {'cupt_code': <MajorCuptCode: วท.บ. สาขาวิชาอาหาร โภชนาการ และการกำหนดอาหาร (ภาษาไทย ปกติ)>, 'criterias': {}}}, ('10020101500101A', ''): {'data': {'major_cupt_code': {'program_code': '10020101500101A', 'program_type': 'ภาษาไทย ปกติ', 'program_type_code': 'A', 'faculty': 2, 'major_code': '', 'title': 'วท.บ. สาขาวิชาเทคโนโลยีระบบเกษตร', 'major_title': '', 'component_weight_type': ''}, 'criterias': {}}, 'objects': {'cupt_code': <MajorCuptCode: วท.บ. สาขาวิชาเทคโนโลยีระบบเกษตร (ภาษาไทย ปกติ)>, 'criterias': {}}}, ('10020101500201A', ''): {'data': {'major_cupt_code': {'program_code': '10020101500201A', 'program_type': 'ภาษาไทย ปกติ', 'program_type_code': 'A', 'faculty': 2, 'major_

In [15]:
cupt_code

<MajorCuptCode: วศ.บ. สาขาวิชาวิศวกรรมการบินและอวกาศ (Double Degree - นานาชาติ พิเศษ)>

In [16]:
def cb(d1,d2):
    res = {}
    for k in d1:
        res[k] = d1[k]
    for k in d2:
        res[k] = d2[k]
    return res

def serialize_admission_criteria(criteria):
    data1 = serializers.serialize('json',[criteria])
    data2 = json.loads(data1)[0]
    
    scores = list(criteria.scorecriteria_set.all())
    score_data1 = serializers.serialize('json', scores)
    score_data2 = json.loads(score_data1)
    
    result = {
        'admission_criteria': data2['fields'],
        'score_criterias': [cb(d['fields'],{'pk': d['pk']}) for d in score_data2],
    }

    #print('data',score_data2)
    
    #print(result)
    return result

In [17]:
#c = 0

for p,m in criterias:
    k = (p,m)
    cupt_code = criterias[k]['objects']['cupt_code']
    print(cupt_code)
    curriculum_majors = CurriculumMajor.objects.filter(cupt_code=cupt_code)
    for m in curriculum_majors:
        if m.admission_project_id not in project_ids:
            continue

        #print(m.admission_project_id)
        major_admission_criterias = CurriculumMajorAdmissionCriteria.objects.filter(curriculum_major=m)
        
        criterias[k]['objects']['criterias'][m.admission_project_id] = [
            {'slots':cmac.slots,
             'admission_criteria': cmac.admission_criteria}
            for cmac in major_admission_criterias
            if cmac.admission_criteria.is_deleted == False
        ]
        criterias[k]['data']['criterias'][m.admission_project_id] = [
            {'slots':cmac.slots,
             'admission_criteria': serialize_admission_criteria(cmac.admission_criteria)}
            for cmac in major_admission_criterias
            if cmac.admission_criteria.is_deleted == False
        ]
        
    #print('------------------')
    #print()
print('done')

วท.บ. สาขาวิชาอาหาร โภชนาการ และการกำหนดอาหาร (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาเทคโนโลยีระบบเกษตร (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาการจัดการศัตรูพืชและสัตว์ (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาสัตวศาสตร์อุตสาหกรรม (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาเคมีการเกษตร (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาวิทยาศาสตร์เกษตร (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาเกษตรเขตร้อน (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาคหกรรมศาสตร์ (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาประมง (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาวนศาสตร์ (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาคณิตศาสตร์ (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาเคมี (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาเคมีอุตสาหกรรม (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาจุลชีววิทยา (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาชีวเคมี (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาชีววิทยา (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาชีววิทยา วิชาเอกสัตววิทยา (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาวิทยาศาสตร์ชีวภาพรังสี (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาวิทยาศาสตร์พื้นพิภพ (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาพฤกษศาสตร์ (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาพันธุศาสตร์ (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาฟิสิกส์ (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาวิทยาศาสตร์นิวเคลีย

วศ.บ. สาขาวิชาวิศวกรรมเครื่องกลและการออกแบบ (ภาษาไทย ปกติ)
วศ.บ. สาขาวิชาวิศวกรรมไฟฟ้าและอิเล็กทรอนิกส์ (ภาษาไทย ปกติ)
วศ.บ. สาขาวิชาวิศวกรรมโยธา (ภาษาไทย ปกติ)
วศ.บ. สาขาวิชาวิศวกรรมอุตสาหการและระบบ (ภาษาไทย ปกติ)
วศ.บ. สาขาวิชาวิศวกรรมระบบการผลิตดิจิทัล (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาอาหารปลอดภัยและโภชนาการ (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาเทคโนโลยีการอาหาร (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาพืชศาสตร์ (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาทรัพยากรเกษตรและการจัดการการผลิต (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาสัตวศาสตร์ (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาเคมีประยุกต์ (ภาษาไทย ปกติ)
วท.บ. สาขาวิชาวิทยาการคอมพิวเตอร์ (ภาษาไทย ปกติ)
วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์ (ภาษาไทย ปกติ)
วศ.บ. สาขาวิชาวิศวกรรมเครื่องกลและการผลิต (ภาษาไทย ปกติ)
วศ.บ. สาขาวิชาวิศวกรรมไฟฟ้า (ภาษาไทย ปกติ)
วศ.บ. สาขาวิชาวิศวกรรมโยธา (ภาษาไทย ปกติ)
วศ.บ. สาขาวิชาวิศวกรรมสิ่งแวดล้อมเพื่อการพัฒนาอย่างยั่งยืน (ภาษาไทย ปกติ)
วศ.บ. สาขาวิชาวิศวกรรมอุตสาหการ (ภาษาไทย ปกติ)
บธ.บ. สาขาวิชาการเงิน (ภาษาไทย ปกติ)
บธ.บ. สาขาวิชาการจัดการ (ภาษาไทย ปกติ)
บช.บ. (ภา

In [18]:
final_data_to_json = {f'{p}-{m}': criterias[(p,m)]['data'] for p,m in criterias.keys()}

In [20]:
filename = 'export-to-portal-66.json'

with open(filename,'w') as f:
    print(json.dumps(final_data_to_json, indent=1),
          file=f)

In [ ]:
print(json.dumps(final_data_to_json, indent=1))